In [2]:
pip install OWSLib



     -------------------------------------- 233.1/233.1 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 508.0/508.0 kB 4.5 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     ---------------------------------------- 3.8/3.8 MB 11.0 MB/s eta 0:00:00
     ---------------------------------------- 153.2/153.2 kB ? eta 0:00:00
     ---------------------------------------- 70.4/70.4 kB ? eta 0:00:00
     ---------------------------------------- 100.2/100.2 kB ? eta 0:00:00
     ---------------------------------------- 124.2/124.2 kB ? eta 0:00:00


In [4]:
pip install pillow


     ---------------------------------------- 2.5/2.5 MB 6.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from owslib.wms import WebMapService
from PIL import Image
import requests
from io import BytesIO

## Python Code zur Nutzung des WMS-Servers

In dem folgenden Abschnitt wird der Web Map Service (WMS) von Bremen verwendet, um Karten- und Geodaten zu laden und anzuzeigen. Der Code stellt eine Verbindung zum WMS-Server her, listet verfügbare Layer auf und ruft Informationen zu einem bestimmten Layer ab.

In [6]:
# WMS-URL: Die URL, die den Web Map Service des WMS-Servers angibt.
wms_url = "https://geodienste.bremen.de/wms_dop10_2023?language=ger"

# Verbindung zum Server herstellen
wms = WebMapService(wms_url)

# Verfügbare Layer anzeigen
# Der WMS-Server enthält eine Reihe von Karten-Layern, die wir durchlaufen und ausgeben können.
# In diesem Fall listen wir die Namen der Layer und deren Titel auf.
for layer in wms.contents:
    print(layer, wms[layer].title)

# Informationen über einen bestimmten Layer abrufen
# In diesem Schritt wählen wir einen bestimmten Layer (DOP10_2023_HB) aus und holen uns seine Metadaten.
layer_name = "DOP10_2023_HB"
layer_info = wms[layer_name]
# Bounding Box des Layers: Hier erhalten wir die geographischen Grenzen (Eckkoordinaten) des Layers.
print("Bounding Box Layer:", layer_info.boundingBox)
# Unterstützte CRS (Koordinatenreferenzsysteme): Der WMS-Server kann verschiedene Koordinatensysteme unterstützen. 
# Wir listen die unterstützten CRS für den ausgewählten Layer auf.
print("Supported CRS:", layer_info.crsOptions)

DOP10 2023 Land Bremen DOP10 2023 Land Bremen
DOP10_2023_HB DOP10 2023 Bremen
DOP10_2023_BHV DOP10 2023 Bremerhaven
Bounding Box Layer: (463942.0, 5873090.0, 500928.0, 5898080.0, 'EPSG:25832')
Supported CRS: ['EPSG:3857', 'EPSG:4326', 'EPSG:4258', 'EPSG:31467', 'EPSG:25833', 'CRS:84', 'EPSG:25832', 'EPSG:3044']


## Funktion zum Herunterladen eines Bildes vom WMS-Server

In dieser Funktion wird ein Bild von einem WMS-Server heruntergeladen. Die Funktion verwendet die `getmap`-Methode des WMS-Servers, um das Bild in einem gewünschten Format und einer gewünschten Größe abzurufen und es auf der Festplatte zu speichern.

In [7]:
def download_image(wms, layer, bbox, size, format, filename):
    response = wms.getmap(layers=[layer],   # Wählen des Layers, das heruntergeladen werden soll
                          styles=[''],      # Stil für das Layer (leer bedeutet standardmäßig)
                          srs='EPSG:25832', # Koordinatenreferenzsystem
                          bbox=bbox,        # Geographische Begrenzung (Bounding Box) des Bildes
                          size=size,        # Größe des Bildes in Pixeln (Breite, Höhe)
                          format=format,    # Format des heruntergeladenen Bildes (z.B. 'image/png')
                          transparent=True) # Option für transparente Hintergründe (True oder False)
    
    with open(filename, 'wb') as out:
        out.write(response.read())
    print(f"{filename} erfolgreich heruntergeladen.")

## Python-Code zum Herunterladen von Kartenbildern von einem WMS-Server

Dieser Code zeigt, wie man mit einem WMS-Server Bilder für ein gesamtes Gebiet oder in kleineren Kacheln herunterladen kann. Die Kacheln sind nützlich, um große Karten in handlichere Teile zu unterteilen, die leichter verarbeitet oder angezeigt werden können.

# 320x320 Pixel

In [8]:

import os
from owslib.wms import WebMapService
from PIL import Image
from io import BytesIO

def download_image(wms, layer, bbox, size, format, filename):
    response = wms.getmap(
        layers=[layer],
        srs='EPSG:25832',
        bbox=bbox,
        size=size,
        format=format,
        transparent=True
    )
    img = Image.open(BytesIO(response.read()))
    img.save(filename)

# WMS-Server-URL
url = "https://geodienste.bremen.de/wms_dop10_2023"

# Verbindung zum WMS-Server herstellen
wms = WebMapService(url, version='1.3.0')

# Layer auswählen
layer = 'DOP10_2023_HB'

# Bounding Box definieren (minx, miny, maxx, maxy)
bbox = (488608.7545, 5881989.4935, 490215.9319, 5883625.7616)

# Verzeichnis für die Ausgabe erstellen
output_dir = "wms_output_320x320"
os.makedirs(output_dir, exist_ok=True)

# Zielgröße der Kacheln in Pixeln
tile_size = 320

# Zielauflösung in Metern pro Pixel
resolution = 0.1  # Beispielwert, bitte anpassen

# Breite und Höhe der Bounding Box in Metern
bbox_width = bbox[2] - bbox[0]
bbox_height = bbox[3] - bbox[1]

# Anzahl der Kacheln in x- und y-Richtung
num_tiles_x = int(bbox_width / (tile_size * resolution))
num_tiles_y = int(bbox_height / (tile_size * resolution))

# Breite und Höhe jeder Kachel in Metern
tile_width = bbox_width / num_tiles_x
tile_height = bbox_height / num_tiles_y


for i in range(num_tiles_y):
    for j in range(num_tiles_x):
        tile_bbox = (
            bbox[0] + j * tile_width,
            bbox[1] + i * tile_height,
            bbox[0] + (j + 1) * tile_width,
            bbox[1] + (i + 1) * tile_height
        )
        
        tile_filename = f"tile_{int(tile_bbox[0])}_{int(tile_bbox[1])}"
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/tiff', f"{output_dir}/{tile_filename}_tiff.tiff")
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/png', f"{output_dir}/{tile_filename}_png.png")

print("Alle Bilder wurden erfolgreich heruntergeladen.")


Maximale Anzahl von Bildern erreicht.
Die ersten 10 Bilder wurden erfolgreich heruntergeladen.
